In [5]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

from binance.client import Client
import matplotlib.pyplot as plt

from binance import ThreadedWebsocketManager

#import talib as ta
import pandas_ta as ta
import time
import pickle
from xgboost import XGBClassifier
import time
#from twilio.rest import Client as TC

import warnings
warnings.filterwarnings("ignore")

In [6]:
class LongShortTrader():
    
    def __init__(self, symbol, bar_length, model, units, position = 0):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units
        self.position = position
        self.trades = 0 
        self.trade_values = []
        
        #*****************add strategy-specific attributes here******************
        self.model = model
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, 
                                interval = self.bar_length,
                                days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
        
        # stop trading session
        if self.trades >= 3: # stop stream after 5 trades
            self.twm.stop()
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL AND STOP")
                self.position = 0
            elif self.position == -1:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL AND STOP")
                self.position = 0
            else: 
                print("STOP")
            
            # send results via sms
            # account_sid = 'AC324ba0a3439404b09cc42cb41cab1ea3'
            # auth_token = '43403305ee82527af3cb9e6dc6e33308'
            # twilio_client = TC(account_sid, auth_token)
            # message = twilio_client.messages.create(
            #                      body=f"CumProfits = {np.sum(self.cum_profits)}$ for {self.symbol}",
            #                      from_='+18608095918',
            #                      to='+49 1517 5020095')
            
            # # send results via sms
            # account_sid = 'AC5a5fed2c33a0239b975dec4756db1da2'
            # auth_token = '93c00768bbab49b952b8edda528747ed'
            # twilio_client = TC(account_sid, auth_token)
            # message = twilio_client.messages.create(
            #                      body=f"CumProfits = {np.sum(self.cum_profits)}$ for {self.symbol}",
            #                      from_='+19896621884',
            #                      to='+49 176 80592510')
            
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades()
        
    def define_strategy(self):
        
        df = self.data.copy()
        
        # Features
        df['ADX'] = ta.ADX(df['High'].values, df['Low'].values, df['Close'].values, timeperiod=5)
        df['RSI'] = ta.RSI(df['Close'].values, timeperiod=5)
        df['SMA'] = ta.SMA(df['Close'].values, timeperiod=10)
        
        predictors_list = ['ADX', 'RSI', 'SMA']

        df["position"] = self.model.predict(df[predictors_list])
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL")
                time.sleep(0.1)
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            elif self.position == -1:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            self.position = 0
        if self.prepared_data["position"].iloc[-1] == -1: # if position is short -> go/stay short
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING SHORT") 
            elif self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL")
                time.sleep(0.1)
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
    
    def report_trade(self, order, going): 
        
        # extract data from order object
        side = order["side"]
        time = pd.to_datetime(order["transactTime"], unit = "ms")
        base_units = float(order["executedQty"])
        quote_units = float(order["cummulativeQuoteQty"])
        price = round(quote_units / base_units, 5)
        
        # calculate trading profits
        self.trades += 1
        if side == "BUY":
            self.trade_values.append(-quote_units)
        elif side == "SELL":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            self.cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            self.cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, self.cum_profits))
        print(100 * "-" + "\n")

In [12]:
import json
path = "../../../config.json"
with open(path) as data:
    config = json.load(data) 
data.close()

print(config)


{'key': 'NgbQ0JPZCgCc9KEfaEaBIhE1AIlSGWYKbGpPUjsxyyviq2TxfJnKRpJC3TlopKOi', 'secret': 'h71b7rQ7QUbYQd2oiR3DKQuwrhJw29i6yqDgDc4s16qdxnTFJnRE1giCamW8169W', 'key_test': 'WGNi2nhFu9FnBfYknpG8qUx3gqWTWynbxowPEHyze3Y104S0ZfGJW3xsh9e5xMWC', 'secret_test': 'tezY53N0GwLCpSXGIoGczdN7y0iu0M4iAxsG0MGz1zNTvTa1qJorPKaZYEJPLHx8', 'MONGO_PASSWORD': 'sTIhAzn9UMlPtrrm', 'MONGO_USER': 'johtorr'}


In [13]:
api_key = config['key_test']
secret_key = config['secret_test']

client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = True)


In [ ]:
if __name__ == "__main__": # only if we run trader.py as a script, please do the following:


    xgb_classifier = pickle.load(open("xgb_clf.pkl", "rb"))
    
    symbol = "BTCUSDT"
    bar_length = "1m"
    model = xgb_classifier
    units = 0.01
    position = 0

    trader = LongShortTrader(symbol = symbol, bar_length = bar_length, model = model, units = units, position = position)

    trader.start_trading(historical_days = 1/24)